In [1]:
import personalities_with_nationality
import prompts
from dotenv import load_dotenv
import os
from openai import OpenAI
from pprint import pprint
import pandas as pd
import asyncio
from openai import AsyncOpenAI
from pprint import pprint
from typing import List, Tuple
load_dotenv()

True

In [2]:
api_key = os.environ["DEEPSEEK_KEY"]

In [3]:

def get_answers(system_prompts, user_prompts):
    assert len(system_prompts) == len(user_prompts)
    client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")


    results_values = list()
    for i in range(len(system_prompts)):
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": system_prompts[i]},
                {"role": "user", "content": user_prompts[i]},
            ],temperature=0,
    max_tokens=5)
        try:
            pprint(f"Result {i+1}: {response.choices[0].message.content}")
            pprint(f"Result {i+1}: {int(response.choices[0].message.content)}")
            results_values.append(int(response.choices[0].message.content))
        except Exception as e:
            results_values.append(None)
    return results_values

In [4]:
import os
import pandas as pd

def process_dataframe(parquet_file):
    """
    Process a parquet dataframe row by row with incremental saves after each cell processing.
    
    Args:
        parquet_file (str): Path to the input parquet file
        output_dir (str): Directory to save incremental processed files
    
    Returns:
        pandas.DataFrame: Final processed dataframe
    """
    
    # Read the parquet file
    df = pd.read_parquet(parquet_file, engine='pyarrow')

    target_columns = ['far_right_EN', 'far_right_PL', 'far_right_RU', 'mod_cons_EN', 'mod_cons_PL', 'mod_cons_RU',
       'prog_left_EN', 'prog_left_PL', 'prog_left_RU', 'centrist_EN', 'centrist_PL', 'centrist_RU']
    
    
    # Process rows one by one
    for index, row in df.iterrows():
        print(f'Line: {index}')
        system_prompts = list()
        user_prompts = list()
        cols_to_process = list()
        for target_col in target_columns:
            # Skip if cell is None already processed
            if not pd.isna(row['Deepseek_v3_' + target_col + '_answer']):
                continue
            
            try:
                cols_to_process.append(target_col)
                
                
                if target_col[-2:] == 'EN':
                    system_prompts.append(prompts.system_prompt_EN)
                elif target_col[-2:] == 'PL':
                    system_prompts.append(prompts.system_prompt_PL)
                elif target_col[-2:] == 'RU':
                    system_prompts.append(prompts.system_prompt_RU)
                else:
                    raise Exception
                
                if target_col[-2:] == 'EN':
                    user_prompts.append(prompts.user_text_EN.replace('{personality}', getattr(personalities_with_nationality, target_col)).replace('{tweet}', row['Text_EN']))
                elif target_col[-2:] == 'PL':
                    user_prompts.append(prompts.user_text_EN.replace('{personality}', getattr(personalities_with_nationality, target_col)).replace('{tweet}', row['Text_PL']))
                elif target_col[-2:] == 'RU':
                    user_prompts.append(prompts.user_text_EN.replace('{personality}', getattr(personalities_with_nationality, target_col)).replace('{tweet}', row['Text_RU']))
                else:
                    raise Exception
                
            
            except Exception as e:
                print(f"Error processing {target_col} at index {index}: {str(e)}")

        values = get_answers(system_prompts, user_prompts)
        for i in range(len(cols_to_process)):
            df.at[index, 'Deepseek_v3_' + cols_to_process[i] + '_answer'] = values[i]
            print(f"Processed {cols_to_process[i]} for key {row['Text_EN']} at index {index}: {values[i]}")
            df.to_parquet(parquet_file_path, engine="pyarrow")
                
    
    print("Final processed dataframe saved.")
    
    return df

# Example usage
if __name__ == "__main__":
    # Replace with your actual parquet file path
    parquet_file_path = 'data/Deepseek_v3/data.parquet'
    
    # Process the dataframe with incremental saves
    result_df = process_dataframe(parquet_file_path)

Line: 1
Line: 2
Line: 3
Line: 4
Line: 5
Line: 6
Line: 7
Line: 8
Line: 9
Line: 10
Line: 11
Line: 12
Line: 13
Line: 14
Line: 15
Line: 16
Line: 17
Line: 18
Line: 19
Line: 20
Line: 21
Line: 22
Line: 23
Line: 24
Line: 25
Line: 26
Line: 27
Line: 28
Line: 29
Line: 30
Line: 31
Line: 32
Line: 33
Line: 34
Line: 35
Line: 36
Line: 37
Line: 38
Line: 39
Line: 40
Line: 41
Line: 42
Line: 43
Line: 44
Line: 45
Line: 46
Line: 47
Line: 48
Line: 49
Line: 50
Line: 51
Line: 52
Line: 53
Line: 54
Line: 55
Line: 56
Line: 57
Line: 58
Line: 59
Line: 60
Line: 61
Line: 62
Line: 63
Line: 64
Line: 65
Line: 66
Line: 67
Line: 68
Line: 69
Line: 70
Line: 71
Line: 72
Line: 73
Line: 74
Line: 75
Line: 76
Line: 77
Line: 78
Line: 79
Line: 80
Line: 81
Line: 82
Line: 83
Line: 84
Line: 85
Line: 86
Line: 87
Line: 88
Line: 89
Line: 90
Line: 91
Line: 92
Line: 93
Line: 94
Line: 95
Line: 96
Line: 97
Line: 98
Line: 99
Line: 100
Line: 101
Line: 102
Line: 103
Line: 104
Line: 105
Line: 106
Line: 107
Line: 108
Line: 109
Line: 110
Line: 11

In [5]:
target_columns = ['far_right_EN', 'far_right_PL', 'far_right_RU', 'mod_cons_EN', 'mod_cons_PL', 'mod_cons_RU',
       'prog_left_EN', 'prog_left_PL', 'prog_left_RU', 'centrist_EN', 'centrist_PL', 'centrist_RU']
getattr(personalities_with_nationality, target_columns[0])


'Name: Mark Thompson\nAge: 52\nSex: Male\nNationality: American\nPolitical Group: Far-right conservative\nOutlook: Distrusts mainstream media, believes "cancel culture" suppresses free speech. Views criticism of Republican policies as unfair attacks. Likely to downplay offensiveness in right-wing tweets, but label progressive tweets as \'aggressive\' or \'divisive.\''

In [5]:
data = pd.read_parquet('data/data_with_nationality_1.parquet', engine="pyarrow")
data.head()

,Text_EN,Text_PL,Text_RU,Translation_quality,Deepseek_far_right_EN_reasoning,Deepseek_far_right_EN_answer,Deepseek_far_right_PL_reasoning,Deepseek_far_right_PL_answer,Deepseek_far_right_RU_reasoning,Deepseek_far_right_RU_answer,...,Deepseek_prog_left_PL_reasoning,Deepseek_prog_left_PL_answer,Deepseek_prog_left_RU_reasoning,Deepseek_prog_left_RU_answer,Deepseek_centrist_EN_reasoning,Deepseek_centrist_EN_answer,Deepseek_centrist_PL_reasoning,Deepseek_centrist_PL_answer,Deepseek_centrist_RU_reasoning,Deepseek_centrist_RU_answer
1,<user> <user> <user> <user> Um the Kurds are h...,"<user> <user> <user> <user> Um, Kurdowie pomag...","<user> <user> <user> <user> Эм, курды помогают...",1.0,"Okay, let me try to figure out how Mark Thomps...",1.0,"Okay, let me try to figure out how Mark Thomps...",1.0,"Alright, let's analyze this step by step. The ...",1.0,...,"Okay, let me try to figure this out. The user ...",0.0,"Alright, let's break this down. Aisha Patel is...",0.0,"Okay, let's break down how James Cooper would ...",1.0,"Okay, let's break this down. The user's statem...",1.0,"Хорошо, давай разберемся с этим запросом. Джей...",1.0
2,<user> This earpiece too plus a wire on his sl...,<user> Ta słuchawka też plus przewód na rękawa...,<user> Этот наушник тоже плюс провод на рукава...,1.0,None,NaN,None,NaN,None,NaN,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
3,<user> <user> Please lost you yelling insanely...,"<użytkownik> <użytkownik> Proszę, zgubcie się,...","<user> <user> Пожалуйста, потеряйтесь, безумно...",1.0,None,NaN,None,NaN,None,NaN,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
4,"<user> <user> <user> Just take the L g, or fin...",<user> <user> <user> Po prostu weź L g lub zna...,"<user> <user> <user> Просто возьми L g, или на...",0.0,None,NaN,None,NaN,None,NaN,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
5,Elections have consequences 101: - Republican...,Wybory mają konsekwencje 101: - Republikańscy ...,Выборы имеют последствия 101: - избиратели-рес...,1.0,None,NaN,None,NaN,None,NaN,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN


In [8]:
from pprint import pprint
pprint(data.iloc[0, 6])

('Okay, let me try to figure out how Mark Thompson would label this statement. '
 "First, the user's statement mentions Kurds helping Ukrainians resist Putin "
 'and the Russian military. Then it criticizes Trump for cutting off aid to '
 'Ukraine to pressure their president for election help, leading to his '
 'impeachment.\n'
 '\n'
 'Mark is a far-right conservative who distrusts mainstream media and sees '
 'criticism of right-wing policies as unfair attacks. He probably downplays '
 'offensive right-wing tweets but calls progressive ones aggressive. So, the '
 'statement criticizes Trump, a right-wing figure, and supports the idea that '
 "his actions were impeachable. From Mark's viewpoint, this could be seen as "
 'an unfair attack on a right-wing leader, thus labeling it as offensive (1). '
 "He might also dismiss the part about Kurds if it's against Russian actions, "
 'but the main issue is the Trump criticism. So, the answer is likely 1.')
